In [ ]:
from alphazero_agent import AlphaZeroAgent
import gymnasium as gym 
import tensorflow as tf
import numpy as np
import gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
env = gym.make("gym_envs/TicTacToe-v0")
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
env = gym.wrappers.FrameStack(env, 4)

config = {
        'activation': 'relu',
        'kernel_initializer': 'orthogonal',
        'optimizer_function': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.2,
        'adam_epsilon': 1e-7,
        'clipnorm': 0.5,
        # NORMALIZATION?
        # REWARD CLIPPING
        'num_epochs': 30,
        'num_filters': 32,
        'kernel_size': 3,
        'stride': 1,
        'num_res_blocks': 5,
        'critic_conv_filters': 32,
        'critic_conv_layers': 1,
        'critic_dense_size': 32,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32,
        'actor_conv_layers': 1,
        'actor_dense_size': 32,
        'actor_dense_layers': 1,
        'memory_size': 180,
        'max_game_length': 9,
        'batch_size': 9,
        'dirichlet_alpha': 0.3,
        'dirichlet_epsilon': 0.25,
        'c_puct': 1,
        'monte_carlo_simulations': 18,
        'two_player': True,
        'weight_decay': 1e-4,
    }    



In [ ]:
array = np.zeros((3, 9))
array[0] = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(array)

In [ ]:
import gymnasium as gym
env = gym.make('gym_envs/TicTacToe-v0')

state, info = env.reset()
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
env.render()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
env.render()
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(8)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(5)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
env.render()


env.reset()
state, reward, terminated, truncated, info = env.step(0)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(3)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(7)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(4)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(2)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(6)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
state, reward, terminated, truncated, info = env.step(1)
print(state)
print("Turn: ", state[2][0][0])
print("Legal moves: ", info['legal_moves'])
print("Terminated:", terminated)
print("Truncated:", truncated)
print("Reward:", reward)



In [1]:
from alphazero_agent import AlphaZeroAgent
from agent_configs import AlphaZeroConfig
from game_configs import TicTacToeConfig

# from alphazero_agent import AlphaZeroAgent
import gymnasium as gym
import tensorflow as tf
import numpy as np
import custom_gym_envs

class ClipReward(gym.RewardWrapper):
    def __init__(self, env, min_reward, max_reward):
        super().__init__(env)
        self.min_reward = min_reward
        self.max_reward = max_reward
        self.reward_range = (min_reward, max_reward)

    def reward(self, reward):
        return np.clip(reward, self.min_reward, self.max_reward)


# env = ClipReward(gym.wrappers.AtariPreprocessing(gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array"), terminal_on_life_loss=True), -1, 1) # as recommended by the original paper, should already include max pooling
# env = TicTacToeEnv(render_mode="rgb_array")
# env = gym.make("MsPacmanNoFrameskip-v4", render_mode="rgb_array")
# env = gym.wrappers.FrameStack(env, 4)
env = gym.make('custom_gym_envs/Connect4-v0', render_mode="rgb_array")


# MODEL SEEMS TO BE UNDERFITTING SO TRY AND GET IT TO OVERFIT THEN FIND A HAPPY MEDIUM
# 1. INCREASE THE NUMBER OF RESIDUAL BLOCKS
# 2. INCREASE THE NUMBER OF FILTERS
# 3. DECREASE REGULARIZATION
# 4. TRY DECREASING LEARNING RATE (maybe its that whole thing where the policy goes to like 1 0 0 0 0... etc and then goes back on the third training step, so maybe the learning rate is too high)
# 5. TO OVERFIT USE LESS DATA (but that is probably just a bad idea)
# config = {
#         'activation': 'relu',
#         'kernel_initializer': 'glorot_uniform',
#         'optimizer': tf.keras.optimizers.legacy.Adam,
#         'learning_rate': 0.001, # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
#         'adam_epsilon': 3.25e-6,
#         'clipnorm': None,
#         # NORMALIZATION?
#         # REWARD CLIPPING
#         'training_steps': 40,
#         'num_filters': 256,
#         'kernel_size': 3,
#         'stride': 1,
#         'num_res_blocks': 20,
#         'critic_conv_filters': 32, # 1
#         'critic_conv_layers': 1,
#         'critic_dense_size': 256,
#         'critic_dense_layers': 1,
#         'actor_conv_filters': 32, # 
#         'actor_conv_layers': 1,
#         'actor_dense_size': 0,
#         'actor_dense_layers': 0,
#         'replay_buffer_size': 800, # IN GAMES
#         'replay_batch_size': 50, # IN MOVES
#         'root_dirichlet_alpha': 0.5, # 2 in theory?
#         'root_exploration_fraction': 0, # 0.25 in paper
#         'pb_c_base': 500,
#         'pb_c_init': 2,
#         'num_simulations': 200,
#         # 'two_player': True,
#         'weight_decay': 0.00, # could try setting this to something other than 0 and increasing learning rate
#         'num_sampling_moves': 0, 
#         'initial_temperature': 1,
#         'exploitation_temperature': 0.1,
#         'value_loss_factor': 1, # could try setting this to something other than 1
#         'games_per_generation': 10, # times 8 from augmentation
#     }

config = {
        'activation': 'relu',
        'kernel_initializer': 'glorot_uniform',
        'optimizer': tf.keras.optimizers.legacy.Adam,
        'learning_rate': 0.0005, #0.0001 # 0.00001 could maybe increase by a factor of 10 or 100 and try to do some weights regularization
        'number_of_lr_cycles': 1, # this will determine the step size based on training steps
        # STILL ADD A SCHEDULE FOR BASE LEARNING RATE (MIN LEARNING RATE)
        'adam_epsilon': 3.25e-6,
        'clipnorm': None,
        # NORMALIZATION?
        # REWARD CLIPPING
        'training_steps': 100, # alpha zero did 700,000, the lessons from alpha zero did 40 generations but 1000 batches per generation, so 40,000 batches (they just had a cyclical learning rate per generation (also they trained twice on the same data every generation))
        'num_filters': 256,
        'kernel_size': 3,
        'stride': 1,
        'residual_blocks': 20,
        'critic_conv_filters': 32, # 1
        'critic_conv_layers': 1,
        'critic_dense_size': 256,
        'critic_dense_layers': 1,
        'actor_conv_filters': 32, # 
        'actor_conv_layers': 1,
        'actor_dense_size': 0,
        'actor_dense_layers': 0,
        'replay_buffer_size': 100, # IN GAMES
        'minibatch_size': 24, # SHOULD BE ROUGHLY SAME AS AVERAGE MOVE PER GENERATION (SO LIKE 7 TIMES NUMBER OF GAMES PLAYED PER GENERATION) <- what was used in the original paper (they played 44M games, 50 moves per game and sampled 700,000 minibatches of size 4096 (so thats like sampling 1 time per move roughly but this was also happening with parrallel data collection i believe))
        'games_per_generation': 1, # times 8 from augmentation
        'root_dirichlet_alpha': 2.5, # Less than 1 more random, greater than one more flat # 2 in theory? # 0.3 in alphazero for chess # TRY CHANGING (MAYBE LOWER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN <- so we want a lesss flat distribution maybe)
        'root_exploration_fraction': 0.25, # 0.25 in paper
        'pb_c_base': 20000, # Seems unimportant to be honest (increases puct the more simulations there are)
        'pb_c_init': 1.25, # 1.25 in paper # MAYBE HIGHER? (IT SEEMS TO PLAY THE SAME LINE OVER AND OVER AGAIN)
        'num_simulations': 50, # INCREASE THIS since the model is missing 1 move wins (and also 2 and 3 move wins (it wins by luck)))
        # 'two_player': True,
        'weight_decay': 0.00001, # could try setting this to something other than 0 and increasing learning rate
        'num_sampling_moves': 30, 
        'exploration_temperature': 1,
        'exploitation_temperature': 0.1,
        'value_loss_factor': 1, # could try setting this to something other than 1
    }

config = AlphaZeroConfig(config, TicTacToeConfig())

agent = AlphaZeroAgent(env, config, "alphazero")

/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1 Physical GPUs, 1 Logical GPUs
Box(0.0, 1.0, (6, 7, 3), float64)
Using optimizer: <class 'keras.src.optimizers.legacy.adam.Adam'>
Using adam_epsilon: 3.25e-06
Using learning_rate: 0.0005
Using clipnorm: None
Using default training_iterations: 1
Using default num_minibatches: 1
Using minibatch_size: 24
Using replay_buffer_size: 100
Using default min_replay_buffer_size: 0
Using training_steps: 100
Using activation: relu
Using kernel_initializer: glorot_uniform
Using kernel_size: 3
Using num_filters: 256
Using residual_blocks: 20
Using critic_conv_layers: 1
Using critic_conv_filters: 32
Using critic_dense_layers: 1
Using critic_dense_size: 256
Using actor_conv_layers: 1
Using actor_conv_filters: 32
Using actor_dense_layers: 0
Using actor_dense_size: 0
Using games_per_generation: 1
Using value_loss_factor: 1
Using weight_decay: 1e-05
Using root_dirichlet_alpha: 2.5
Using root_exploration_fraction: 0.25
Using num_simulations: 50
Using num_sampling_moves: 30
Using exploration_temperature: 1

2024-03-30 17:14:03.115549: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-30 17:14:03.115576: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-30 17:14:03.115580: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-30 17:14:03.115616: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-30 17:14:03.115636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamont

In [2]:
agent.checkpoint_interval = 1
agent.train()

Training Step  1


/Users/jonathanlamontange-kratz/Library/Python/3.9/lib/python/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Predicted Policy  [0.14285715 0.14285715 0.14285715 0.14285715 0.14285715 0.14285715
 0.14285715]
Predicted Value  0.0
Target Policy [0.14 0.14 0.14 0.14 0.12 0.16 0.16]
Action  5
Predicted Policy  [0.14276    0.1429839  0.14246327 0.14259376 0.1432516  0.14284351
 0.14310393]
Predicted Value  -0.0031319891568273306
Target Policy [0.14       0.18000001 0.14       0.12       0.14       0.12
 0.16      ]
Action  4
Predicted Policy  [0.14283921 0.14286415 0.14280201 0.14279085 0.14294146 0.1428796
 0.14288269]
Predicted Value  -0.0005694590508937836
Target Policy [0.14 0.16 0.14 0.14 0.14 0.16 0.12]
Action  6
Predicted Policy  [0.14274615 0.14300048 0.1424631  0.14260793 0.1432159  0.14286523
 0.1431012 ]
Predicted Value  -0.0030530039221048355
Target Policy [0.12 0.14 0.16 0.16 0.14 0.14 0.14]
Action  4
Predicted Policy  [0.14286307 0.14288493 0.1427208  0.1427533  0.1429949  0.14289762
 0.14288539]
Predicted Value  -0.0006869083736091852
Target Policy [0.14 0.14 0.16 0.14 0.14 0.14 0.14

KeyboardInterrupt: 

In [ ]:
agent.model.load_weights("./alphazero.keras")
agent.train()